In [1]:
import pandas as pd
import numpy as np
import csv
#from pandasql import PandaSQL
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# File Directory
- All files are in the folder __proj1__
- __ICD_9gem__    is the raw data downloaded from [CMS.gov](https://www.cms.gov/Medicare/Coding/ICD10/Downloads/2016-General-Equivalence-Mappings.zip)
- __ICD_10gem__   is the raw data downloaded from [CMS.gov](https://www.cms.gov/Medicare/Coding/ICD10/Downloads/2016-General-Equivalence-Mappings.zip)
- __ICD_9_desc__  is the raw data downloaded from [CMS.gov](https://www.cms.gov/Medicare/Coding/ICD9ProviderDiagnosticCodes/Downloads/ICD-9-CM-v32-master-descriptions.zip)
- __ICD_10_desc__ is the raw data downloaded from [CMS.gov](https://www.cms.gov/Medicare/Coding/ICD10/Downloads/2016-Code-Descriptions-in-Tabular-Order.zip)
- __ICD9_Walk__   is the previous ICD9 crosswalk done by Christina.
- __ICD10_Walk__  is the previous ICD10 crosswalk done by Christina.


In [2]:
ICD_9Gem_loc    = r".\2016-General-Equivalence-Mappings\2016_I9gem.txt"
ICD_10Gem_loc   = r".\2016-General-Equivalence-Mappings\2016_I10gem.txt"
ICD_9_desc_loc  = r".\ICD-9-CM-v32-master-descriptions\CMS32_DESC_LONG_SHORT_DX.xlsx"
ICD_10_desc_loc = r".\2016-Code-Descriptions-in-Tabular-Order\icd10cm_order_2016.txt"
ICD9_Walk_loc   = r".\ICD9_Walk.txt"
ICD10_Walk_loc  = r".\ICD10_Walk.txt"

### Load Data

In [3]:
ICD_9gem    = pd.read_fwf(ICD_9Gem_loc,    colspecs=[(0,5),(6,13),(14,19)], header = None, names = ['ICD9','ICD10','flags'], dtype='object')
ICD_10gem   = pd.read_fwf(ICD_10Gem_loc,   colspecs=[(0,7),(8,13),(14,19)], header = None, names = ['ICD10','ICD9','flags'], dtype='object')
ICD_9_desc  = pd.read_excel(ICD_9_desc_loc)
ICD_10_desc = pd.read_fwf(ICD_10_desc_loc, colspecs=[(0,5),(6,13),(14,15),(16,76),(77,300)], header = None, names = ['Order_number','ICD10','is_HIPAA_covered','ICD10_Desc','ICD10_long_Desc'], dtype='object')
ICD9_Walk   = pd.read_table(ICD9_Walk_loc,sep=',',dtype='object')
ICD10_Walk  = pd.read_table(ICD10_Walk_loc,sep=',',dtype='object')

In [4]:
ICD10_Walk.head()

,ICD10,ICD10_Desc,ICD9,ICD9_Desc,APPROX,NOMAP,COMBO,SCENARIO,CHOICE,CATEGORY,OBSOLETE
0,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol...",001.0,Cholera d/t vib cholerae,0,0,0,0,0,A00,0
1,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor",001.1,Cholera d/t vib el tor,0,0,0,0,0,A00,0
2,A00.9,"Cholera, unspecified",001.9,Cholera NOS,0,0,0,0,0,A00,0
3,A01.00,"Typhoid fever, unspecified",002.0,Typhoid fever,1,0,0,0,0,A01,0
4,A01.01,Typhoid meningitis,002.0,Typhoid fever,1,0,0,0,0,A01,0


# Reproduce ICD10_walk
1. add ICD10 short description to the raw data
2. add ICD9 short description to the raw data
3. split the flag field into 5 columns which represents following information:
    - _APPROX_ : the translation is not a precise equivalent(1)
    - _NOMAP_ : there is no corresponding code in the target system(1)
    - _COMBO_ : more than one code is the target system is required to satisfy the meaning of the code in the source system(1)
    - _SCENARIO_ : in a combination entry, a collection of codes from the target system containing the necessary codes that combined as directed will satisfy the equivalent meaning of a code in the source system(1-9)
    - _CHOICE_ : in a combination entry, a list of one or more codes in the target system from which one code must be chosen to satisfy the equivalent meaning of a code in the source system(1-9)
4. change ICD10 codes to the dot form
    - if the code is more than 3 characters long then add "." at index 3
5. change ICD9 codes to the dot form
    - if the code is more than 3 characters long then add "." at index 3 except for those start with "E". if the code starts with "E" and is more than 4 characters long, then add "." at index 4

In [5]:
#add ICD10 short description
ICD10_walk = pd.merge(ICD_10gem,
                      ICD_10_desc,
                      how='left',
                      on = 'ICD10')

ICD10_walk = ICD10_walk.drop(['Order_number',
                              'is_HIPAA_covered',
                              'ICD10_long_Desc'],
                             axis = 1)

#add ICD9 short description
ICD10_walk.ICD9[ICD10_walk.ICD9 == 'NoDx'] = np.nan
ICD10_walk = pd.merge(ICD10_walk,
                      ICD_9_desc,
                      how='left',
                      left_on='ICD9',
                      right_on='DIAGNOSIS CODE')

ICD10_walk.drop(['DIAGNOSIS CODE',
                 'LONG DESCRIPTION'],
                axis=1,
                inplace=True)


#rename ICD9 description column
ICD10_walk.rename(columns = {'SHORT DESCRIPTION': 'ICD9_Desc'}, inplace = True)

#split the flag into 5 columns
ICD10_walk["APPROX"]  = ICD10_walk["flags"].str.slice(0,1)
ICD10_walk["NOMAP"]   = ICD10_walk["flags"].str.slice(1,2)
ICD10_walk["COMBO"]   = ICD10_walk["flags"].str.slice(2,3)
ICD10_walk["SCENARIO"]= ICD10_walk["flags"].str.slice(3,4)
ICD10_walk["CHOICE"]  = ICD10_walk["flags"].str.slice(4,5)
ICD10_walk.drop('flags', axis = 1, inplace = True)
 
#change ICD10 code to dot form
ICD10_walk.ICD10 = ICD10_walk.ICD10.map(lambda x: x[0:3] + '.' + x[3:] if len(x) > 3 else x)

#change ICD9 code to dot form
def addDot(x):
    '''
    ICD9 codes with prefix "E": dots are at index position 4, no dots if length is not greater than 4
    ICD9 codes not with prefix "E": dots are at index position 3
    '''
    if (not pd.isnull(x)) and x[0] != 'E':
        if len(x) == 3:
            return x
        else:
            return( x[0:3] + '.' + x[3:])
    elif pd.isnull(x):
        return(x)
    else:
        if len(x) == 4:
            return x
        else:
            return(x[0:4] + '.' + x[4:])

ICD10_walk.ICD9 = ICD10_walk.ICD9.map(lambda x: addDot(x))

#rearrange column order
ICD10_walk = ICD10_walk[['ICD10',
                         'ICD10_Desc',
                         'ICD9',
                         'ICD9_Desc',
                         'APPROX',
                         'NOMAP',
                         'COMBO',
                         'SCENARIO',
                         'CHOICE']]
ICD10_walk.to_csv(r'.\ICD10_2016_walk.csv',index=False)

### Test if two versions of crosswalk tables are the same
- sort 'ICD9' and 'ICD10' columns in both tables

In [6]:
#test if the merged table is the same as the correct one
ICD10_walk_sorted = ICD10_walk.sort_values(['ICD9', 'ICD10'])
ICD10_Walk_sorted = ICD10_Walk.sort_values(['ICD9', 'ICD10'])
ICD10_walk_sorted.reset_index(drop = True, inplace = True)
ICD10_Walk_sorted.reset_index(drop = True, inplace = True)

#pd.testing.assert_frame_equal(ICD10_walk_sorted,ICD10_Walk_sorted.iloc[:,:9])

In [7]:
A = ICD10_walk_sorted
B = ICD10_Walk_sorted.iloc[:,:9]

In [8]:
#check if any unmatched records in merged table
C = pd.merge(A, B, how='left', on=['ICD10', 'ICD9'])
C = C[C.ICD10_Desc_y.isnull()]
C.head()

,ICD10,ICD10_Desc_x,ICD9,ICD9_Desc_x,APPROX_x,NOMAP_x,COMBO_x,SCENARIO_x,CHOICE_x,ICD10_Desc_y,ICD9_Desc_y,APPROX_y,NOMAP_y,COMBO_y,SCENARIO_y,CHOICE_y
79188,R40.241,Glasgow coma scale score 13-15,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79189,R40.242,Glasgow coma scale score 9-12,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79190,R40.243,Glasgow coma scale score 3-8,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79191,R40.244,"Oth coma,w/o Glasgow coma scale score,or w/par...",NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
C.to_csv(r".\ICD10_Compare.txt", sep=',', quoting=csv.QUOTE_ALL, index=False)

In [10]:
#check if any discrepancy in matched records
C = pd.merge(A, B, how='inner', on=['ICD10', 'ICD9'])
C = C[((C.ICD10_Desc_x != C.ICD10_Desc_y) | (C.ICD9_Desc_x != C.ICD9_Desc_y)) & (~C.ICD9_Desc_x.isnull())]
C.to_csv(r".\ICD10_Compare2.txt", sep=',', quoting=csv.QUOTE_ALL, index=False)

# Reproduce ICD9_walk
- follow the same procedure as above

In [11]:
#################################################
#           create ICD9_walk
#################################################

#add ICD9 short description
ICD9_walk = pd.merge(ICD_9gem,
                     ICD_9_desc,
                     how='left',
                     left_on='ICD9',
                     right_on='DIAGNOSIS CODE')

ICD9_walk = ICD9_walk.drop(['DIAGNOSIS CODE',
                            'LONG DESCRIPTION'],
                           axis = 1)

#add ICD10 short description
ICD9_walk.ICD10[ICD9_walk.ICD10 == 'NoDx'] = np.nan
ICD9_walk = pd.merge(ICD9_walk,
                     ICD_10_desc,
                     how='left',
                     on='ICD10')

ICD9_walk.drop(['Order_number',
                'is_HIPAA_covered',
                'ICD10_long_Desc'],
               axis=1,
               inplace=True)

#rename ICD9 description column
ICD9_walk.rename(columns = {'SHORT DESCRIPTION': 'ICD9_Desc'},inplace = True)

#split the flag into 5 columns
ICD9_walk["APPROX"]  = ICD9_walk["flags"].str.slice(0,1)
ICD9_walk["NOMAP"]   = ICD9_walk["flags"].str.slice(1,2)
ICD9_walk["COMBO"]   = ICD9_walk["flags"].str.slice(2,3)
ICD9_walk["SCENARIO"]= ICD9_walk["flags"].str.slice(3,4)
ICD9_walk["CHOICE"]  = ICD9_walk["flags"].str.slice(4,5)
ICD9_walk.drop('flags', axis = 1, inplace = True)

#change ICD10 code to dot form
def addDot2(x):
    if pd.isnull(x):
        return(x)
    else:
        if len(x)>3:
            return(x[0:3] + '.' + x[3:])
        else:
            return x
        
ICD9_walk.ICD10 = ICD9_walk.ICD10.map(lambda x: addDot2(x))

#change ICD9 code to dot form
#for codes start with "E" put dot at index 4, other codes at index 3
ICD9_walk.ICD9 = ICD9_walk.ICD9.map(lambda x: addDot(x))

#rearrange column order
ICD9_walk = ICD9_walk[['ICD9',
                       'ICD9_Desc',
                       'ICD10',
                       'ICD10_Desc',
                       'APPROX',
                       'NOMAP',
                       'COMBO',
                       'SCENARIO',
                       'CHOICE']]
ICD9_walk.to_csv(r'.\ICD9_2016_walk.csv',index=False)

### Test if two versions of crosswalk tables are the same
- sort 'ICD9' and 'ICD10' columns in both tables

In [12]:
#test if the merged table is the same as the correct one
ICD9_walk_sorted = ICD9_walk.sort_values(['ICD9', 'ICD10'])
ICD9_Walk_sorted = ICD9_Walk.sort_values(['ICD9', 'ICD10'])
ICD9_walk_sorted.reset_index(drop = True, inplace = True)
ICD9_Walk_sorted.reset_index(drop = True, inplace = True)

A = ICD9_walk_sorted
B = ICD9_Walk_sorted.iloc[:,:9]

In [13]:
#check if any unmatched records in merged table
C = pd.merge(A, B, how='left', on=['ICD9', 'ICD10'])
C = C[C.ICD9_Desc_y.isnull()]
C.head()

,ICD9,ICD9_Desc_x,ICD10,ICD10_Desc_x,APPROX_x,NOMAP_x,COMBO_x,SCENARIO_x,CHOICE_x,ICD9_Desc_y,ICD10_Desc_y,APPROX_y,NOMAP_y,COMBO_y,SCENARIO_y,CHOICE_y
11873,764.09,Lt-for-dates 2500+g,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11892,764.19,Lt-for-date w/mal 2500+g,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20637,E927.0,Overxrt-sudn stren mvmt,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20638,E927.1,Overxrt-prolng stc postn,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20640,E927.3,Cumltv trma-repetv motn,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
C.to_csv(r".\ICD9_Compare.txt", sep=',', quoting=csv.QUOTE_ALL, index=False)

In [15]:
#check if any discrepancy in matched records
C = pd.merge(A, B, how='inner', on=['ICD9', 'ICD10'])
C = C[((C.ICD10_Desc_x != C.ICD10_Desc_y) | (C.ICD9_Desc_x != C.ICD9_Desc_y)) & (~C.ICD10_Desc_x.isnull())]
C.to_csv(r".\ICD9_Compare2.txt", sep=',', quoting=csv.QUOTE_ALL, index=False)

# Get ICD9 and ICD10 Category Tables
- __ICD10_category__ : use the first 3 charaters as the category code
- __ICD9_category__ : if the ICD9 code has corresponding ICD10 code(s), then use the first 3 characters of the first matching ICD10 code as the category code, else assign category blank

In [16]:
#######################################
#      get ICD10 code category
#######################################

#select two useful columns from ICD10_walk
ICD10_category = ICD10_walk.loc[:, ['ICD10', 'ICD10_Desc']]
ICD10_category.drop_duplicates(subset = ['ICD10'], inplace = True)

#use first 3 characters as category for ICD10 codes
ICD10_category['Category'] = ICD10_category.ICD10.str.slice(0,3)

#left join to get category description
ICD10_category = pd.merge(ICD10_category,
                          ICD_10_desc.loc[:,['ICD10','ICD10_Desc']],
                          left_on='Category',
                          right_on='ICD10',
                          how='left')
#trim the dataframe
ICD10_category.drop(['ICD10_y'],axis = 1,inplace=True)
ICD10_category.columns = ['ICD10',
                          'ICD10_Desc',
                          'Category',
                          'Category_Desc']
ICD10_category.head()

,ICD10,ICD10_Desc,Category,Category_Desc
0,A00.0,"Cholera due to Vibrio cholerae 01, biovar chol...",A00,Cholera
1,A00.1,"Cholera due to Vibrio cholerae 01, biovar eltor",A00,Cholera
2,A00.9,"Cholera, unspecified",A00,Cholera
3,A01.00,"Typhoid fever, unspecified",A01,Typhoid and paratyphoid fevers
4,A01.01,Typhoid meningitis,A01,Typhoid and paratyphoid fevers


In [17]:
########################################
#         get ICD9 code category
########################################

#select three useful columns from ICD9_walk
ICD9_category = ICD9_walk.loc[:, ['ICD9', 'ICD9_Desc', 'ICD10']]
#some ICD9 codes map to multiple ICD10 codes, use the first one as its category
ICD9_category=  ICD9_category.groupby('ICD9').nth(0)
ICD9_category.reset_index(inplace = True)

#left join ICD10 category table to get category and category descrioption
ICD9_category = pd.merge(ICD9_category,
                         ICD10_category[['ICD10','Category','Category_Desc']],
                         on='ICD10',
                         how='left')
#trim the dataframe
ICD9_category.drop('ICD10',axis=1,inplace = True)
ICD9_category.head()

,ICD9,ICD9_Desc,Category,Category_Desc
0,001.0,Cholera d/t vib cholerae,A00,Cholera
1,001.1,Cholera d/t vib el tor,A00,Cholera
2,001.9,Cholera NOS,A00,Cholera
3,002.0,Typhoid fever,A01,Typhoid and paratyphoid fevers
4,002.1,Paratyphoid fever a,A01,Typhoid and paratyphoid fevers


In [18]:
ICD10_category.to_csv(r'.\ICD10_2016_category.csv',index=False)
ICD9_category.to_csv(r'.\ICD9_2016_category.csv',index=False)

In [19]:
ICD10_walk[(ICD10_walk.ICD10=='R40.241') & (ICD10_walk.APPROX=='1')]

,ICD10,ICD10_Desc,ICD9,ICD9_Desc,APPROX,NOMAP,COMBO,SCENARIO,CHOICE
26654,R40.241,Glasgow coma scale score 13-15,NaN,NaN,1,1,0,0,0
